# 001_Initial Watch Records.ipynb

___

## 개요

**문제의식**  

'몇 시간씩이나 보는지, 무엇을 보는 지 눈으로 확인해도 이렇게 계속 볼까?'


**문제 확인**  

채널 소유주의 수익 창출에 대한 시청자 데이터는 제공하지만,  
일반 이용자는 모바일에서 일주일 간의 시청 그래프만 확인 가능


**목표**  

[1] 시청 기록 페이지의 정보를 수집  
[2] 시청 패턴을 발견할 수 있는 시각화 대시보드 만들기


**진행 단계**  

[1] 지정한 날짜까지의 데이터 수집 도구 프로그래밍 (bs4, undetected_chromedriver, Youtube Data API 활용)  


**해결 중인 문제**  

[1] 자료형 (Tableau, SQL, Python 간) 결정 및 변환  
[2] 분석 방법 결정 

___

### 현재까지 수집된 데이터셋

In [2]:
import pandas as pd

def read_csv(directory):
    df = pd.read_csv(directory, encoding='utf-8')
    df.drop('Unnamed: 0', axis=1, inplace=True)
    return df


In [3]:
df_record = read_csv('data/20230329_watch_record_initial.csv')
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account
0,Digimon Adventure tri - Warp Evolution,Dogecoin Live,5:41,2023-03-28,오후 6:29,1,Film & Animation,0.13,kjaehw0207
1,[디지몬 프론티어 세계관 정리] 최악의 어둠의 투사가 진정한 모습을 찾게 되는 과정...,엘로아비디오 EloaVideo,8:19,2023-03-28,오후 6:21,1,Film & Animation,1.00,kjaehw0207
2,기억은 잃었지만 개같이 진화[디지몬 어드벤처 트라이 4장 : 상실],애니무비,12:02,2023-03-28,오후 6:20,1,Film & Animation,0.10,kjaehw0207
3,[자막뉴스] 뒤돌아서 한눈판 일본 관광객...172번 버스기사님의 직감 / KBS ...,KBS News,1:43,2023-03-28,오후 6:16,25,News & Politics,1.00,kjaehw0207
4,트위치스트리머쇼메쇼메하이라이트1,Dplus KIA,8:03,2023-03-28,오후 6:12,20,Gaming,0.32,kjaehw0207


In [5]:
df_search = read_csv('data/20230329_search_record_initial.csv')
df_search.head()


,searchword,search_date,search_time,account
0,https://www.youtube.com/watch?v=zeoaJiLdxEo,2023-03-16,오전 1:32,kjaehw0207
1,https://www.youtube.com/watch?v=N7PeqQm3wGw,2023-03-15,오전 12:41,kjaehw0207
2,https://www.youtube.com/watch?v=VKZQzTZW5lo,2023-03-11,오전 1:13,kjaehw0207
3,https://www.youtube.com/watch?v=z6dIglvdrvc,2023-03-11,오전 12:57,kjaehw0207
4,https://www.youtube.com/watch?v=ks59EyAaWDs,2023-03-11,오전 12:50,kjaehw0207


In [6]:
df_search['searchword'].unique()[30:40]

array(['인공지능 모델 문제 정의', '인공지능 문제 정의', 'fourplay snowbound', '랄로',
       'dota openai', 'lck', 'woghks.study', '모두의연구소',
       'https://www.youtube.com/watch?v=jKONrQ3ZEKE', '생활코딩'],
      dtype=object)

## 1. Web Access

### 1.1. Modules

In [ ]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크 (https://github.com/ultrafunkamsterdam/undetected-chromedriver)


In [6]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC

import undetected_chromedriver as uc

from bs4 import BeautifulSoup

import time


### 1.2. Web Access

In [47]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')

dir_file_account = f"secure/id_pw_{2}.txt"


#### 1.2.1. 계정 정보 호출 및 입력

In [48]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
try:
    btn_sign_in = driver.find_element(
        By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')
    btn_sign_in.click()
except:
    pass

# 계정 이메일 입력
try:
    txt = open(dir_file_account).readlines()
    account = txt[0].split(' ')[1].rstrip('\n')

    input_account = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
    input_account.send_keys(account)
except:
    pass
finally:  # 개인정보 보호를 위해 변수 txt 제거
    del txt

# [다음] 버튼 클릭해 패스워드 입력 단계으로 전환
try:
    btn_next_password = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
    btn_next_password.click()
except:
    pass


#### 1.2.2. 비밀번호 호출 및 입력 (후 삭제)

In [49]:
# 계정 비밀번호 입력

try:
    txt = open(dir_file_account).readlines()
    password = txt[1].split(' ')[1].rstrip('\n')

    input_password = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
    input_password.send_keys(password)
except:
    pass
finally:  # 개인정보 보호를 위해 변수 txt, password 제거
    del password
    del txt

# [다음] 버튼 입력해 로그인 클릭
try:
    btn_next_sign_in = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
    btn_next_sign_in.click()
except:
    pass


#### 1.2.3. 연락처 확인을 통한 개인 맞춤 설정 페이지 팝업 시 처리

In [50]:
try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


In [51]:
driver.refresh()


## 2. Data Definition

#### 2.1. Modules

In [22]:
import pandas as pd
from datetime import datetime

# API 실행에 필요한 모듈
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow


#### 2.2. Variables

In [17]:
# Variables (1) 시청기록 태그 속성


# 기본 영상 정보
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>


# 시청비율 판별 정보
nm_class_thumbnail = "OUPWA"  # 기록 우측에 있는 영상 정보 (썸네일) / tag = <div>
nm_class_percentage = "HmLFgd"  # 썸네일 하단 빨간 Percentage Bar / tag = <div>


# 시청기록 - video id
# replace(pattern, '')로 id 값 추출
id_str_pattern = "https://www.youtube.com/watch?v="


'\nWatchDate = container[data-date] / c-wiz 태그의 [data-date] 값\n시청비율 확인을 위해 nm_class_RunningTime 활용\n'

In [19]:
# Variables (2) 유튜브 데이터 API


# API 정보, API Key => API build
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
DEVELOPER_KEY = open('secure/api_key.txt').readline()

youtube = build(API_SERVICE_NAME, API_VERSION, developerKey=DEVELOPER_KEY)


#### 2.3. Functions

In [24]:
# Function (1) - datetime 객체, 문자열 변환 관련


def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d')


get_date('20230101')


datetime.datetime(2023, 1, 1, 0, 0)

In [25]:
# Function (2) - Category Id & Title

""" Google API - Youtube DATA API 활용
    세부 query
    - videos().list(): id 값(watch?v="여기 표시되는 값")으로 동영상 정보 반환 -> 내부의 categoryId 호출
    - videoCategories().list(): categoryId 값(1~44)으로 카테고리 정보 반환 -> 내부의 categoryTitle 호출
"""

""" 개선 필요
        categoryTitle이 영어로 호출됨
        query 옵션에서 한국어를 호출하거나, 번역해야 한다.
"""


def get_catId(input_id):
    video = youtube.videos().list(
        part='snippet',
        id=input_id
    ).execute()

    return video['items'][0]['snippet']['categoryId']


def get_catTitle(num):
    catId = youtube.videoCategories().list(
        id=num,
        part='snippet'
    ).execute()

    try:
        result = catId['items'][0]['snippet']['title']
    except:
        result = ''

    return result


In [26]:
# Function (3) - Watched_YN classifier

""" 구현 가능 Option (1)
    - 구현법
        1. 현재 레코드의 watch_time과 이전 레코드의 watch_time의 차이값 구한다
        2. 영상 길이와 비슷하면 watched, 아닐 시 not_watched

    - 문제점
        1. 마지막으로 시청한 영상은 차이를 구할 수 없다.
        2. 길이 5분 이상 광고를 방치한 경우 시간 계산에 오류가 생긴다
"""

""" 구현 가능 Option (2) -> 채택
    - 구현법
        1. 시청 기록 레코드의 썸네일의 진행도 막대를 활용한다.
        2. 시청 완료하지 않은 영상에 표시되며, 태그 내부의 시청 비율 값을 추출한다.

    - 문제점
        1. 라이브 스트리밍의 경우 진행도 막대가 출력되지 않는다.
        2. 대충 넘기기만 해도 진행도 막대가 표시된다. 
"""


def classify_watched(content):
    running_time = content.find('div', class_=nm_class_RunningTime).text
    thumbnail = content.find('div', class_=nm_class_thumbnail)
    percentage = thumbnail.find('div', class_=nm_class_percentage)

    try:
        result = int(percentage['style'].split(":")[1][:-1]) / 100

    except:
        if len(running_time) > 5:  # 영상 길이가 1시간 이상(0:00:00)인 경우
            result = 0  # 미시청으로 간주
        else:
            result = 1  # 시청으로 간주

    return result


#### 2.4. Empty DataFrame

In [29]:
columns_record = ['title', 'channel', 'running_time',
                  'watch_date', 'watch_time', 'category_id', 'category_title', 'watched (%)', 'account']
df_record = pd.DataFrame(columns=columns_record)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account


In [38]:
columns_search = ['searchword', 'search_date', 'search_time', 'account']
df_search = pd.DataFrame(columns=columns_search)
df_search.head()


,searchword,search_date,search_time,account


## 3. Data Collection

### 3.1. 입력 날짜까지의 시청 기록 호출 (03/28 진행 - 미해결)

#### 3.1.1. Functions

In [31]:
# 날짜 입력 함수

def get_date_data():
    "4자리 연도, 1~2자리 월/일 숫자를 입력받아 8자리 문자열로 출력. 오입력된 경우 다시 입력하도록 함"
    while True:
        try:
            year, month, day = [int(input(f"{field}를(을) 입력해주세요")) for field in (
                "연도(4자리)", "월(1~2자리)", "일(1~2자리)")]
            if not (1000 <= year <= 9999 and 1 <= month <= 12 and 1 <= day <= 31):
                input("Invalid input. Wanna Try again?")
            else:
                break
        except:
            print("Invalid input. Wanna Try again?")

    return f"{year:04d}{month:02d}{day:02d}"


print(f'함수 Test. 입력된 날짜는 {get_date_data()}입니다.')


함수 Test. 입력된 날짜는 20220831입니다.


In [32]:
# 날짜 변환 함수

def convert_date(str_date):
    "'3월 28일'과 같은 형태의 날짜를 8자리 문자열 데이터로 출력"
    nums = [int(each[:-1]) for each in str_date.split(' ')]

    if len(nums) == 2:  # 연도가 생략된 올해 날짜
        year = datetime.strftime(datetime.today(), '%Y')
        return f"{year}{nums[0]:02d}{nums[1]:02d}"

    else:
        return f"{nums[0]}{nums[1]:02d}{nums[2]:02d}"


print(f"Test1. {convert_date('9월 8일')}")
print(f"Test2. {convert_date('2018년 8월 5일')}")


Test1. 20230908
Test2. 20180805


In [33]:
# 현재 호출된 레코드의 마지막 날짜 확인

def get_last_date():
    nm_class_DateDivider = 'MCZgpb'  # 날짜 구분 막대

    contents = BeautifulSoup(driver.page_source)
    date_dividers = contents.find_all('div', nm_class_DateDivider)

    return int(convert_date(date_dividers[-1].text))


#### 3.1.2. Pre-work (1) 입력 날짜까지의 레코드 호출

In [52]:
# 입력 날짜가 확인될 때까지 End Key 반복 입력

target_date = int(get_date_data())
last_date = int(get_last_date())


while last_date >= target_date:

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//body'))).send_keys(Keys.END)

    last_date = get_last_date()

else:
    pass

# 날짜 확인
target_date, last_date


#### 3.1.3. Pre-work (2) 입력 날짜까지의 레코드로 한정

**문제**

target_date에 시청 기록이 없으면 어떻게 해야 할까?  
target_date까지의 기록만을 수집하려면 어떻게 해야 할까?

여기도 마찬가지. 콘텐츠 로딩이 일찍 끝나면 어떻게 해줄 건가?

In [54]:
contents = BeautifulSoup(driver.page_source)

containers = [each for each in contents.find('div', jsname='bN97Pc').children]


### 3.2. Html Parsing → 데이터프레임에 입력

In [56]:
# 레코드 분류 및 카테고리/시청여부 입력

soup = BeautifulSoup(driver.page_source)
contents = soup.find_all('c-wiz', class_='xDtZAf')

idx_rec = len(df_record)  # df_record의 idx
idx_ser = len(df_search)  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime):  # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")
            try:
                title = content.find('a', class_=nm_class_title).text
                channel = content.find('div', class_=nm_class_channel).text
                running_time = content.find(
                    'div', class_=nm_class_RunningTime).text
                watch_date = get_date(content['data-date'])
                watch_time = ' '.join(
                    (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
                watched = classify_watched(content)

                id = content.find('a')['href'].replace(id_str_pattern, '')
                category_id = get_catId(id)
                category_title = get_catTitle(category_id)

                df_record.loc[idx_rec] = [title, channel, running_time,
                                        watch_date, watch_time, category_id, category_title, watched, account]
                idx_rec += 1
            except:
                pass
            
        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time, account]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Watch Record
1: Ad
2: Search Record
3: Watch Record
4: Watch Record
5: Watch Record
6: Watch Record
7: Watch Record
8: Watch Record
9: Ad
10: Ad
11: Search Record
12: Watch Record
13: Watch Record
14: Watch Record
15: Ad
16: Watch Record
17: Watch Record
18: Watch Record
19: Watch Record
20: Watch Record
21: Watch Record
22: Watch Record
23: Ad
24: Watch Record
25: Watch Record
26: Watch Record
27: Ad
28: Watch Record
29: Watch Record
30: Watch Record
31: Watch Record
32: Watch Record
33: Watch Record
34: Watch Record
35: Ad
36: Watch Record
37: Watch Record
38: Watch Record
39: Watch Record
40: Watch Record
41: Ad
42: Search Record
43: Search Record
44: Watch Record
45: Watch Record
46: Watch Record
47: Search Record
48: Ad
49: Ad
50: Ad
51: Watch Record
52: Ad
53: Ad
54: Watch Record
55: Ad
56: Ad
57: Watch Record
58: Search Record
59: Watch Record
60: Ad
61: Search Record
62: Watch Record
63: Watch Record
64: Watch Record
65: Watch Record
66: Watch Record
67: Ad
68: Watch Record


**문제** 

1. 삭제된 영상에 대한 레코드는? 에러를 내면서 멈춰버린다.
태그 어디에서 문제가 생겨서 그런 것인지 확인하자.

2. 음악 들으면 레코드가 빨리 쌓인다.
단점: API에 데이터를 요청하는 데 시간/비용이 더 많이 든다
장점: Youtube를 음악 감상에 사용하는 지 또한 사용 패턴으로 해석할 여지가 생긴다.

In [ ]:
driver.close()


### 3.3. Dataframe 확인 및 .csv 파일로 저장

#### 3.3.1. 시청 기록 데이터 - 입력 여부 확인 (head, tail) 및 저장

In [59]:
df_record.head(2)

,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account
0,Digimon Adventure tri - Warp Evolution,Dogecoin Live,5:41,2023-03-28,오후 6:29,1,Film & Animation,0.13,kjaehw0207
1,[디지몬 프론티어 세계관 정리] 최악의 어둠의 투사가 진정한 모습을 찾게 되는 과정...,엘로아비디오 EloaVideo,8:19,2023-03-28,오후 6:21,1,Film & Animation,1.00,kjaehw0207


In [60]:
df_record.tail(2)

,title,channel,running_time,watch_date,watch_time,category_id,category_title,watched (%),account
2277,"""전 국민에 학습 휴가""...대학 문턱 더 낮춘다 / YTN",YTN,2:29,2022-12-29,오후 11:51,25,News & Politics,0.55,woghks.study
2278,ENG) 비전공자 개발자 취준생의 밤을 본 현직 개발자의 반응 [모두가 잠든 밤],스튜디오V [ STUDIO V ],13:05,2022-12-29,오후 11:35,24,Entertainment,1.00,woghks.study


In [69]:
df_record.to_csv(f"data/{containers[0]['data-date']}_watch_record_initial.csv", encoding='utf-8')
df_record.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2279 entries, 0 to 2278
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           2279 non-null   object        
 1   channel         2279 non-null   object        
 2   running_time    2279 non-null   object        
 3   watch_date      2279 non-null   datetime64[ns]
 4   watch_time      2279 non-null   object        
 5   category_id     2279 non-null   object        
 6   category_title  2279 non-null   object        
 7   watched (%)     2279 non-null   float64       
 8   account         2279 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 178.0+ KB


#### 3.3.2. 검색 기록 데이터 - 입력 여부 확인 (head, tail) 및 저장

In [64]:
df_search.head(2)

,searchword,search_date,search_time,account
0,https://www.youtube.com/watch?v=zeoaJiLdxEo,2023-03-16,오전 1:32,kjaehw0207
1,https://www.youtube.com/watch?v=N7PeqQm3wGw,2023-03-15,오전 12:41,kjaehw0207


In [65]:
df_search.tail(2)

,searchword,search_date,search_time,account
319,https://www.youtube.com/watch?v=KDxgxengEYs,2022-12-28,오후 11:30,woghks.study
320,https://www.youtube.com/watch?v=6w_blk3zvwE,2022-12-28,오후 11:11,woghks.study


In [70]:
df_search.to_csv(f"data/{containers[0]['data-date']}_search_record_initial.csv", encoding='utf-8')
df_search.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 321 entries, 0 to 320
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   searchword   321 non-null    object        
 1   search_date  321 non-null    datetime64[ns]
 2   search_time  321 non-null    object        
 3   account      321 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 12.5+ KB


## 4. Notes (라 쓰고 Code recycle bin으로 읽는다)

In [ ]:
# 3-3) 입력한 날짜 이전의 레코드 호출

target_date = get_date_data()

btn_calendar = driver.find_element(
    By.XPATH, '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/div/div[2]/div[2]/span/div[2]/c-wiz[1]/div/div/div/div/div[1]/button')
btn_calendar.click()

input_date = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input')))
input_date.send_keys(target_date)

btn_apply = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/div[9]/div/div[2]/span/div[2]/div/c-wiz/div/div[4]/div/div/div[2]/div/button/div[3]')))
btn_apply.click()


In [153]:
# 3-3) 입력한 개수만큼 레코드 호출

import math
num_target_records = 3000
# (오늘 날짜 - 목표 날짜) * 일평균 시청 영상 갯수 * 2.5를 10단위 올림한 수

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
num_current_records = len(soup.find_all('c-wiz', class_='xDtZAf'))


# int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))

num_press_end = int(
    math.ceil((num_target_records - num_current_records) / 100))

for n in range(num_press_end):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(2)
